In [8]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [14]:
# Load the Dataset

train_features = pd.read_csv('../data/train_features.csv')
test_features = pd.read_csv('../data/test_features.csv')
train_labels = pd.read_csv('../data/train_labels.csv')
label_cols = train_labels.columns[1:]
df = train_features.merge(train_labels, on='id')

In [5]:
# Train test split the data (70-30 percent split)

train_df, val_df = train_test_split(df, test_size=0.3, random_state=2)

In [ ]:
#Define Model parameters:

IMG_SIZE = 128
BATCH_SIZE = 24
base = '../data'

In [12]:
def decode_row(filepath, label):
    full_path = tf.strings.join([base, filepath], separator='/')
    img = tf.io.read_file(full_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE]) / 255.0
    return img, tf.cast(label, tf.float32)

def make_ds(df, shuffle=True):
    filepaths = df['filepath'].values.astype(str)
    labels    = df['label_vec'].tolist()

    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(filepaths))
    ds = ds.map(decode_row, num_parallel_calls=tf.data.AUTOTUNE)
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [13]:
train_ds = make_ds(train_df, shuffle=True)
val_ds   = make_ds(val_df, shuffle=False)

## Build a Custom Convolutional Neural Network using Tensorflow

In [16]:
num_classes = len(label_cols)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D( 32, 3, activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D( 64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='sigmoid')
])

model.summary()

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,672 (12.61 MB)

 Trainable params: 3,305,672 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
 79/413 ━━━━━━━━━━━━━━━━━━━━ 17:20 3s/step - accuracy: 0.1847 - auc: 0.5516 - loss: 0.4328

KeyboardInterrupt: 

## More Lightweight faster to train model

In [17]:
num_classes = len(label_cols)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D( 32, 3, activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D( 64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(num_classes, activation='sigmoid')
])

model.summary()

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 94,280 (368.28 KB)

 Trainable params: 94,280 (368.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 1319s 3s/step - accuracy: 0.1502 - auc: 0.5553 - loss: 0.3927 - val_accuracy: 0.2332 - val_auc: 0.6524 - val_loss: 0.3590
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 90s 217ms/step - accuracy: 0.2747 - auc: 0.6856 - loss: 0.3506 - val_accuracy: 0.3132 - val_auc: 0.7368 - val_loss: 0.3345
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 91s 219ms/step - accuracy: 0.3464 - auc: 0.7659 - loss: 0.3228 - val_accuracy: 0.3187 - val_auc: 0.7424 - val_loss: 0.3343
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 94s 227ms/step - accuracy: 0.3579 - auc: 0.7786 - loss: 0.3169 - val_accuracy: 0.3572 - val_auc: 0.7787 - val_loss: 0.3172
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 91s 220ms/step - accuracy: 0.3721 - auc: 0.7887 - loss: 0.3111 - val_accuracy: 0.3608 - val_auc: 0.7834 - val_loss: 0.3187
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 90s 218ms/step - accuracy: 0.3673 - auc: 0.7909 - loss: 0.3098 - val_accuracy: 0.3881 - val_auc: 0.8090 - val_loss: 0.3023
Epoch 7/10
413/413 ━━━━